I'll break down the GWmodel temporal and spatiotemporal distance calculation rules into clear mathematical notation, building from the simplest components to the complete GTWR formula. This will help you understand the mathematical foundation that underlies the code implementation.

## Fundamental Temporal Distance Function

The basic temporal distance between two time points is defined by the `ti.dist` function:

$$d_t(t_1, t_2) = |t_2 - t_1|$$

where $t_1$ and $t_2$ are time values, and the absolute value ensures we always get a positive distance regardless of temporal ordering. This formula applies when both time points are in compatible formats (numeric, integer, or appropriately converted date formats).

## Causal Temporal Distance Vector

The `ti.distv` function introduces the crucial concept of temporal causality. For a focal prediction time $t_f$ and a vector of observation times $\mathbf{t}_{obs} = [t_1, t_2, \ldots, t_n]$, the causal temporal distance vector is defined as:

$$d_t^{causal}(t_f, t_i) = \begin{cases} 
|t_i - t_f| & \text{if } t_f \geq t_i \\
\infty & \text{if } t_f < t_i
\end{cases}$$

This mathematical formulation captures the essential principle that future observations cannot influence past predictions. When $t_f < t_i$ (trying to use future data to predict the past), the distance becomes infinite, effectively giving zero weight to that observation in any subsequent calculations.

## Temporal Distance Matrix Construction

The `ti.distm` function builds a complete temporal distance matrix. For observation times $\mathbf{t}_{obs} = [t_1^{obs}, t_2^{obs}, \ldots, t_n^{obs}]$ and regression times $\mathbf{t}_{reg} = [t_1^{reg}, t_2^{reg}, \ldots, t_m^{reg}]$, the temporal distance matrix $\mathbf{T}$ has dimensions $n \times m$ where:

$$T_{i,j} = d_t^{causal}(t_j^{reg}, t_i^{obs}) = \begin{cases} 
|t_i^{obs} - t_j^{reg}| & \text{if } t_j^{reg} \geq t_i^{obs} \\
\infty & \text{if } t_j^{reg} < t_i^{obs}
\end{cases}$$

The key insight here is that matrix element $T_{i,j}$ represents the temporal distance when using observation $i$ to make a prediction at regression point $j$. This creates an asymmetric matrix structure due to the temporal causality constraint.

## Spatial Distance Foundation

Before combining temporal and spatial components, we need the spatial distance matrix. Using Euclidean distance (the default when $p=2$), for spatial coordinates $\mathbf{s}_i = [x_i, y_i]$ and $\mathbf{s}_j = [x_j, y_j]$:

$$d_s(i,j) = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$$

This creates a symmetric spatial distance matrix $\mathbf{S}$ where $S_{i,j} = S_{j,i}$ represents the spatial separation between locations $i$ and $j$.

## The Complete Spatiotemporal Distance Formula

The `st.dist` function combines spatial and temporal distances using the GTWR spatiotemporal distance formula. For each pair of observations $i$ and $j$, the spatiotemporal distance is:

$$d_{st}(i,j) = \lambda \cdot S_{i,j} + (1-\lambda) \cdot T_{i,j} + 2\sqrt{\lambda(1-\lambda) \cdot S_{i,j} \cdot T_{i,j}} \cdot \cos(\xi)$$

Let me break down each component of this sophisticated formula to help you understand its mathematical behavior:

**Linear Spatial Component**: $\lambda \cdot S_{i,j}$ represents the spatial contribution to the total distance, weighted by the parameter $\lambda \in [0,1]$. When $\lambda = 1$, the model becomes purely spatial (traditional GWR).

**Linear Temporal Component**: $(1-\lambda) \cdot T_{i,j}$ represents the temporal contribution, weighted by $(1-\lambda)$. When $\lambda = 0$, the model becomes purely temporal.

**Interaction Term**: $2\sqrt{\lambda(1-\lambda) \cdot S_{i,j} \cdot T_{i,j}} \cdot \cos(\xi)$ captures the interaction between spatial and temporal separation. This term becomes largest when both spatial and temporal distances are moderate and when $\lambda \approx 0.5$ (balanced spatial-temporal weighting).

## Understanding the Mathematical Behavior

The interaction term creates several important mathematical properties. When either $S_{i,j} = 0$ (same location) or $T_{i,j} = 0$ (same time), the interaction term vanishes, and the distance reduces to purely temporal or spatial respectively. This makes intuitive sense because there cannot be spatiotemporal interaction when one dimension is absent.

The parameter $\xi$ (ksi) controls the nature of the spatial-temporal interaction through the cosine function. When $\xi = 0$, we have $\cos(0) = 1$, creating maximum positive interaction. When $\xi = \pi/2$, we have $\cos(\pi/2) = 0$, eliminating the interaction term entirely.

## Special Cases and Boundary Conditions

Several important boundary conditions emerge from this mathematical framework:

**Same observation ($i = j$)**: Both spatial and temporal distances equal zero, yielding $d_{st}(i,i) = 0$, as expected for the distance from any observation to itself.

**Temporal causality violations**: When $T_{i,j} = \infty$, the entire spatiotemporal distance becomes infinite regardless of spatial proximity, ensuring that future observations never influence past predictions.

**Pure spatial case ($\lambda = 1$)**: The formula reduces to $d_{st}(i,j) = S_{i,j}$, recovering traditional GWR behavior.

**Pure temporal case ($\lambda = 0$)**: The formula reduces to $d_{st}(i,j) = T_{i,j}$, creating a purely time-based weighting scheme.

## Matrix Properties and Computational Implications

The resulting spatiotemporal distance matrix has several important mathematical properties that affect both interpretation and computation. Due to the temporal causality constraint, the matrix is generally asymmetric: $d_{st}(i,j) \neq d_{st}(j,i)$ when observations $i$ and $j$ occur at different times.

The matrix contains approximately half finite entries and half infinite entries, creating a block structure that reflects the temporal ordering of observations. This sparse-like structure (though not technically sparse due to the infinite values) affects both memory requirements and computational strategies for GTWR implementation.

Understanding these mathematical foundations helps explain why GTWR can capture complex spatiotemporal patterns while respecting the fundamental constraint that predictions can only use historically available information. The elegant mathematical framework ensures that spatial proximity, temporal recency, and their interaction all contribute appropriately to the final prediction weights used in the locally weighted regression.

# Understanding GTWR Temporal Distance Through a Complete Table

Let me show you the temporal distance calculation results in a clear, systematic table format. This will help you see exactly how the causality constraint creates the distinctive pattern in GTWR's temporal distance matrix.

## Core Scientific Concept: Temporal Causality in Spatiotemporal Modeling

Before diving into the numbers, let's understand what we're calculating. In GTWR (Geographically and Temporally Weighted Regression), we need to determine how much influence each historical observation should have on predictions at different times. The fundamental principle is **temporal causality** - future events cannot influence past predictions, just like how tomorrow's weather cannot affect yesterday's crop growth.

Think of it like this: imagine you're a detective trying to solve a case that happened on Monday. You can use evidence from Sunday, Monday, and earlier days, but you cannot use evidence from Tuesday or Wednesday because that evidence didn't exist when the Monday event occurred. GTWR applies this same logical constraint to statistical predictions.

## Building the Temporal Distance Matrix Step by Step

### Step 1: Understanding the Data Structure

Your dataset has 16 observations arranged as follows:
- 4 spatial locations (A1, A2, A3, A4)  
- 4 time periods (1, 2, 3, 4)
- Each location observed at each time = 4 × 4 = 16 total observations

### Step 2: The Causality Check Rule

For each pair of observations, we ask: "Can observation i (at time t_i) be used to predict at time t_j?"

**The Mathematical Rule**: If t_j ≥ t_i, then YES (finite distance)
**If t_j < t_i, then NO (infinite distance)**

### Step 3: Distance Calculation When Allowed

When the causality check passes, we calculate: **distance = |t_j - t_i|**

## Complete 16×16 Temporal Distance Matrix

Here's the complete temporal distance matrix. Rows represent "from" observations, columns represent "to" predictions:

| From\To | Row1<br>(A1,t1) | Row2<br>(A1,t2) | Row3<br>(A1,t3) | Row4<br>(A1,t4) | Row5<br>(A2,t1) | Row6<br>(A2,t2) | Row7<br>(A2,t3) | Row8<br>(A2,t4) | Row9<br>(A3,t1) | Row10<br>(A3,t2) | Row11<br>(A3,t3) | Row12<br>(A3,t4) | Row13<br>(A4,t1) | Row14<br>(A4,t2) | Row15<br>(A4,t3) | Row16<br>(A4,t4) |
|---------|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
| **Row1 (A1,t1)** | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 |
| **Row2 (A1,t2)** | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 |
| **Row3 (A1,t3)** | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 |
| **Row4 (A1,t4)** | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 |
| **Row5 (A2,t1)** | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 |
| **Row6 (A2,t2)** | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 |
| **Row7 (A2,t3)** | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 |
| **Row8 (A2,t4)** | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 |
| **Row9 (A3,t1)** | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 |
| **Row10 (A3,t2)** | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 |
| **Row11 (A3,t3)** | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 |
| **Row12 (A3,t4)** | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 |
| **Row13 (A4,t1)** | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 | 0 | 1 | 2 | 3 |
| **Row14 (A4,t2)** | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 | ∞ | 0 | 1 | 2 |
| **Row15 (A4,t3)** | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 | ∞ | ∞ | 0 | 1 |
| **Row16 (A4,t4)** | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 | ∞ | ∞ | ∞ | 0 |

### Step 4: Reading the Table - A Practical Example

Let's trace through how to read this table using concrete examples:

**Example 1**: Row 1 to Column 6
- Row 1 = A1 at time 1 (observation from A1 at time 1)
- Column 6 = A2 at time 2 (predicting for A2 at time 2)
- Question: Can time 1 data predict time 2? YES (2 ≥ 1)
- Distance: |2 - 1| = **1**

**Example 2**: Row 6 to Column 1  
- Row 6 = A2 at time 2 (observation from A2 at time 2)
- Column 1 = A1 at time 1 (predicting for A1 at time 1)
- Question: Can time 2 data predict time 1? NO (1 < 2)
- Distance: **∞** (causality violation)

### Step 5: Recognizing the Beautiful Pattern

Notice the elegant mathematical structure that emerges:

**Block Diagonal Pattern**: The matrix shows clear blocks where finite and infinite values are organized. This isn't random - it's the mathematical expression of time's arrow.

**Temporal Gradient**: Within the allowed (finite) regions, distances increase as temporal separation increases (0, 1, 2, 3), creating a natural weighting where recent observations matter more than distant ones.

**Spatial Independence**: Notice that the temporal distance between A1-time1 and A2-time2 is the same as between A3-time1 and A4-time2. This is correct because temporal distance depends only on time, not spatial location.

## Connecting to the Bigger Picture

This temporal distance matrix serves as one crucial component in GTWR's sophisticated prediction system. When combined with spatial distances through the GTWR formula, it ensures that predictions are both spatially and temporally sensible.

**Key Insight**: The abundance of infinite values (roughly half the matrix) isn't a problem - it's a feature! These infinite distances ensure that future observations get zero weight in predictions, making GTWR respect the fundamental constraint that real-world predictions can only use historically available information.

This mathematical framework allows GTWR to excel in applications like your ground subsidence monitoring, where recent site conditions typically provide more relevant information for current predictions than measurements from many months ago, while still allowing spatially distant but temporally recent observations to contribute valuable information.